# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [2]:
# Hacé la separación en esta celda

X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [3]:
# Esto es lo que hace
import numpy as np
np.random.seed(123)
from sklearn.model_selection import train_test_split
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [4]:
# Creá en esta celda la variable param_grid

param_grid= [
    {'max_depth': [1,2,3,4,5], 'max_features':[1,2,3,4,5]}, 
]

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [5]:
# Importa y crea un GridSearchCV en esta celda
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

tree_reg=DecisionTreeRegressor(random_state=0)
grid_search=GridSearchCV(tree_reg,param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True)


A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [6]:
# Hace el fit de grid search en esta celda

grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=0,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid=[{'max_depth': [1, 2, 3, 4, 5],
                          'max_features': [1, 2, 3, 4, 5]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='n

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [7]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [9]:
# Mostrá los grid_scores en esta celda
grid_search.cv_results_

{'mean_fit_time': array([0.02090015, 0.00554757, 0.00555096, 0.00584707, 0.00565476,
        0.00552697, 0.00574679, 0.00582776, 0.00583968, 0.00589304,
        0.00567431, 0.00578799, 0.00624013, 0.00598326, 0.00647917,
        0.00650249, 0.00654387, 0.00645413, 0.00617752, 0.00750918,
        0.00606451, 0.00607572, 0.01468925, 0.00674939, 0.00726347]),
 'std_fit_time': array([3.05986909e-02, 3.31236778e-05, 2.36386220e-05, 4.35568718e-04,
        1.22877651e-04, 3.45732918e-05, 1.88404896e-04, 1.69760632e-04,
        1.81424514e-04, 7.94015342e-05, 7.36353610e-05, 1.77739258e-04,
        1.90184933e-04, 1.35888183e-04, 6.29147674e-04, 2.07444475e-04,
        5.49671829e-04, 1.34352421e-04, 2.31092555e-04, 1.82736487e-03,
        2.96694410e-04, 4.14186256e-05, 1.68846970e-02, 2.87356607e-04,
        1.14010898e-03]),
 'mean_score_time': array([0.00227237, 0.00188336, 0.00188851, 0.00219359, 0.00198927,
        0.0019031 , 0.00191278, 0.00202856, 0.00192051, 0.00204525,
        0.00

De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

In [58]:
np.sqrt(-grid_search.cv_results_['mean_test_score']) #Por ejemplo en el caso de test 


array([31301.60015297, 31301.68774359, 31103.55841322, 31103.55841322,
       31103.55841322, 31288.05975301, 31242.61115168, 30139.90199528,
       31082.13019503, 31073.55184898, 31262.21166141, 30864.19321895,
       27017.94533905, 29883.2768864 , 30939.75603099, 31141.63952116,
       30638.73390264, 27660.32672719, 29391.95000303, 29627.56184838,
       30472.44956699, 27835.04632453, 27574.30334018, 29343.5798869 ,
       30272.83390009])

In [59]:
np.sqrt(-grid_search.cv_results_['mean_train_score']) #Por ejemplo en el caso de train

array([31289.20270543, 31281.16984535, 31098.47039635, 31098.47039635,
       31098.47039635, 31267.26651979, 31242.58506704, 30046.36128975,
       31032.1463353 , 31043.90262235, 31218.26331608, 30751.90300204,
       26893.59822182, 29607.04974008, 30857.65697992, 30935.98329319,
       30509.67850058, 27435.35226047, 28905.43920222, 29462.31760295,
       30130.16762166, 27277.83037075, 27107.6359727 , 28995.94556341,
       30000.08596049])

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [13]:
# Mostrás los resultados en esta celda
grid_search.best_params_

{'max_depth': 3, 'max_features': 3}

In [60]:
np.sqrt(-grid_search.best_score_)

27017.94533904872

Convertimos a RMSE.

In [19]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [24]:
param_grid2= [
    {"min_samples_split": [2, 10, 20], "max_depth": [None, 2, 5, 10, 15],"min_samples_leaf": [1, 5, 10, 15],"max_leaf_nodes": [None, 5, 10, 20]}, 
]

In [25]:
grid_search2=GridSearchCV(tree_reg,param_grid2,cv=5,scoring='neg_mean_squared_error',return_train_score=True)

In [27]:
grid_search2.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=0,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid=[{'max_depth': [None, 2, 5, 10, 15],
                          'max_leaf_nodes': [None, 5, 10, 20],
                          'min_samples_leaf': [1, 5, 10, 15],
                          'mi

In [32]:
grid_search2.cv_results_

{'mean_fit_time': array([0.04824476, 0.03550076, 0.03224792, 0.03176589, 0.03190565,
        0.02947598, 0.02865462, 0.02778645, 0.02783303, 0.0257966 ,
        0.02576027, 0.0257443 , 0.01411223, 0.01409249, 0.0140883 ,
        0.01411529, 0.01405978, 0.01407614, 0.01407247, 0.01406617,
        0.01404333, 0.01401582, 0.01402855, 0.01405759, 0.01699553,
        0.01722531, 0.01702232, 0.01698332, 0.016997  , 0.01697311,
        0.01700053, 0.01696796, 0.01699796, 0.01694627, 0.01701016,
        0.01695037, 0.01979642, 0.01980023, 0.01985283, 0.01981001,
        0.01978283, 0.01976347, 0.02000036, 0.01969686, 0.01969433,
        0.01961775, 0.01959429, 0.01955934, 0.01075001, 0.01070614,
        0.01076717, 0.01085873, 0.01076198, 0.01075063, 0.01068716,
        0.01069517, 0.01074219, 0.01088448, 0.0107295 , 0.01070771,
        0.01070843, 0.01072631, 0.01073518, 0.0108438 , 0.01075711,
        0.01068463, 0.01073613, 0.01074009, 0.0107038 , 0.01068349,
        0.0107163 , 0.01074634,

In [34]:
grid_search2.best_params_

{'max_depth': 10,
 'max_leaf_nodes': None,
 'min_samples_leaf': 15,
 'min_samples_split': 2}

In [61]:
np.sqrt(-grid_search2.cv_results_['mean_test_score']) #Por ejemplo en el caso de test 

array([26087.15741204, 24329.70058927, 23128.24107276, 22983.77529261,
       22983.77529261, 22281.9855823 , 21904.98244941, 21904.98244941,
       21904.98244941, 21577.12148868, 21577.12148868, 21577.12148868,
       24600.03791514, 24600.03791514, 24600.03791514, 24600.03791514,
       24600.03791514, 24600.03791514, 24600.03791514, 24600.03791514,
       24600.03791514, 24600.03791514, 24600.03791514, 24600.03791514,
       22855.35927591, 22855.35927591, 22855.35927591, 22855.35927591,
       22855.35927591, 22855.35927591, 22855.35927591, 22855.35927591,
       22855.35927591, 22855.35927591, 22855.35927591, 22855.35927591,
       22014.89800988, 22014.89800988, 22014.89800988, 22014.89800988,
       22014.89800988, 22014.89800988, 22018.27305432, 22018.27305432,
       22018.27305432, 22046.43365794, 22046.43365794, 22046.43365794,
       25156.55579937, 25156.55579937, 25156.55579937, 25156.55579937,
       25156.55579937, 25156.55579937, 25156.55579937, 25156.55579937,
      

In [62]:
np.sqrt(-grid_search2.cv_results_['mean_train_score']) #Por ejemplo en el caso de test 

array([  465.63809915, 10776.61493709, 14114.66482189, 13826.68367857,
       13826.68367857, 15612.8544353 , 16671.27216186, 16671.27216186,
       16671.27216186, 17879.82596972, 17879.82596972, 17879.82596972,
       24402.0465702 , 24402.0465702 , 24402.0465702 , 24402.0465702 ,
       24402.0465702 , 24402.0465702 , 24402.0465702 , 24402.0465702 ,
       24402.0465702 , 24402.0465702 , 24402.0465702 , 24402.0465702 ,
       22506.55572136, 22506.55572136, 22506.55572136, 22506.55572136,
       22506.55572136, 22506.55572136, 22506.55572136, 22506.55572136,
       22506.55572136, 22506.55572136, 22506.55572136, 22506.55572136,
       21352.4048556 , 21352.4048556 , 21352.4048556 , 21352.4048556 ,
       21352.4048556 , 21352.4048556 , 21358.78542153, 21358.78542153,
       21358.78542153, 21373.64787813, 21373.64787813, 21373.64787813,
       25019.6562957 , 25019.6562957 , 25019.6562957 , 25019.6562957 ,
       25019.6562957 , 25019.6562957 , 25019.6562957 , 25019.6562957 ,
      

In [63]:
np.sqrt(-grid_search.best_score_)

27017.94533904872

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [20]:
optimised_decision_tree = grid_search.best_estimator_

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [64]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

25820.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [49]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [23]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [50]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,98157.836,18157.836
1,128000.000,131079.617,3079.617
2,150000.000,131079.617,18920.383
3,85000.000,131079.617,46079.617
4,135000.000,133881.445,1118.555
5,135000.000,98157.836,36842.164
6,68000.000,131079.617,63079.617
7,110000.000,131079.617,21079.617
8,134000.000,131079.617,2920.383
9,110000.000,98157.836,11842.164
